### General advice (delete this cell before submitting for review)

> * When choosing a location for your analysis, **select an area that has data on both the NCI and DEA Sandbox** to allow your code to be run on both environments. 
For example, you can check this for Landsat using the [DEA Explorer](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3/1990) (use the drop-down menu to view all products).
As of September 2019, the `DEA Sandbox` has a single year of continental Landsat data for 2015-16, and the full 1987-onward time-series for three locations (Perth WA, Brisbane QLD, and western NSW).
> * When adding **Products used**, embed the hyperlink to that specific product on the DEA Explorer using the `[product_name](product url)` syntax.
> * When writing in Markdown cells, start each sentence on a **new line**.
This makes it easy to see changes through git commits.
> * Use Australian English in markdown cells and code comments.
> * Check the [known issues](https://github.com/GeoscienceAustralia/dea-docs/wiki/Known-issues) for formatting regarding the conversion of notebooks to DEA docs using Sphinx.
Things to be aware of:
    * Sphinx is highly sensitive to bulleted lists:
        * Ensure that there is an empty line between any preceding text and the list
        * Only use the `*` bullet (`-` is not recognised)
        * Sublists must be indented by 4 spaces
    * Two kinds of formatting cannot be used simultaneously:
        * Hyperlinked code: \[\`code_format\`](hyperlink) fails
        * Bolded code: \*\*\`code_format\`\*\* fails
    * Headers must appear in heirachical order (`#`, `##`, `###`, `####`) and there can only be one title (`#`).
> * Use the [PEP8 standard](https://www.python.org/dev/peps/pep-0008/) for code. To make sure all code in the notebook is consistent, you can use the `jupyterlab_code_formatter` tool: select each code cell, then click `Edit` and then one of the `Apply X Formatter` options (`YAPF` or `Black` are recommended). This will reformat the code in the cell to a consistent style.
> * For additional guidance, refer to the style conventions and layouts in approved `develop` branch notebooks. 
Examples include
    * [Frequently_used_code/Using_load_ard.ipynb](./Frequently_used_code/Using_load_ard.ipynb)
    * [Real_world_examples/Coastal_erosion.ipynb](./Real_world_examples/Coastal_erosion.ipynb)
    * [Scripts/dea_datahandling.py](./Scripts/dea_datahandling.py)
> * The DEA Image placed in the title cell will display as long as the notebook is contained in one of the standard directories.
It does not work in the highest level directory (hence why it doesn't display in the original template notebook).
> * In the final notebook cell, include a set of relevant tags which are used to build the DEA User Guide's [Tag Index](https://docs.dea.ga.gov.au/genindex.html). 
Use all lower-case (unless the tag is an acronym), separate words with spaces (unless it is the name of an imported module), and [re-use existing tags](https://github.com/GeoscienceAustralia/dea-notebooks/wiki/List-of-tags).
Ensure the tags cell below is in `Raw` format, rather than `Markdown` or `Code`.


# Exporting Landsat Collection 3 vegetation-related indicies <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Compatability:** Notebook currently compatible with the `NCI`|`DEA Sandbox` environment only
* **Products used:** 
[ga_ls5t_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3),
[ga_ls7e_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls7e_ard_3),
[ga_ls8c_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3)
* **Special requirements:** An _optional_ description of any special requirements, e.g. If running on the [NCI](https://nci.org.au/), ensure that `module load otps` is run prior to launching this notebook
* **Prerequisites:** An _optional_ list of any notebooks that should be run or content that should be understood prior to launching this notebook


## Background
An *optional* overview of the scientific, economic or environmental management issue or challenge being addressed by Digital Earth Australia. 
For `Beginners_Guide` or `Frequently_Used_Code` notebooks, this may include information about why the particular technique or approach is useful or required. 
If you need to cite a scientific paper or link to a website, use a persistent DOI link if possible and link in-text (e.g. [Dhu et al. 2017](https://doi.org/10.1080/20964471.2017.1402490)).

## Description
A _compulsory_ description of the notebook, including a brief overview of how Digital Earth Australia helps to address the problem set out above.
It can be good to include a run-down of the tools/methods that will be demonstrated in the notebook:

1. First we do this
2. Then we do this
3. Finally we do this

***

## Getting started

Provide any particular instructions that the user might need, e.g. To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

Use standard import commands; some are shown below. 
Begin with any `iPython` magic commands, followed by standard Python packages, then any additional functionality you need from the `Scripts` directory.

In [57]:
%matplotlib inline

import sys
import warnings
import matplotlib.pyplot as plt
import calendar
import numpy as np
import xarray as xr

import dask
from dask.utils import parse_bytes
from dask.distributed import Client, LocalCluster

import datacube
from datacube.storage import masking
from datacube.helpers import write_geotiff
from datacube.utils.rio import configure_s3_access
from datacube.utils.dask import start_local_dask

from psutil import virtual_memory, cpu_count

# Load custom DEA notebook functions
sys.path.append('../dea-notebooks/Scripts')
import dea_datahandling
import dea_plotting
import DEADataHandling
from dea_bandindices import calculate_indices

### Connect to the datacube

Connect to the datacube so we can access DEA data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [58]:
try:
    dc_landsat3 = datacube.Datacube(app='VegProductsExport_Coll3', env='c3-samples')
except:
    dc_landsat3 = datacube.Datacube(app='VegProductsExport_Coll3')

### Analysis parameters

* `dry_months`: Specific months of the year that correspond with dry season/low rainfall conditions. Values range from 0-11. 


In [59]:
dry_months = [5,6,7]

### Set up dask cluster
This will help keep our memory use down and conduct the analysis in parallel. If you'd like to view the dask dashboard, click on the hyperlink that prints below the cell.

The parameters for generating the local dask cluster are automatically generated, but if you wish to alter them use the documentation [here](https://distributed.dask.org/en/stable/local-cluster.html). 

Put simply, the code below identifies how many cpus and how much RAM the computer has and generates a local cluster using those variables.

> **Note:** Use this markdown format (sparingly) to draw particular attention to an important point or caveat

In [60]:
# Figure out how much memory/cpu we really have (those are set by jupyterhub)
cpu_limit = cpu_count()
cpu_limit = int(cpu_limit) if cpu_limit > 0 else 4

mem_limit = virtual_memory()
mem_limit = mem_limit.total
mem_limit = mem_limit if mem_limit > 0 else parse_bytes('8Gb')

# leave 3Gb for notebook itself
mem_limit -= parse_bytes('3Gb')

# close previous client if any, so we can re-run this cell without issues
client = locals().get('client', None)
if client is not None:
    client.close()
    del client

# start up a local cluster
client = start_local_dask(n_workers=1,
                          threads_per_worker=cpu_limit,
                          memory_limit=mem_limit)

# show the dask cluster settings
display(client)

/g/data/v10/public/modules/dea-env/20191127/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:37018 Dashboard: http://127.0.0.1:44898/status,Cluster Workers: 1 Cores: 8 Memory: 30.67 GB


### Define spatial and temporal query

If running this notebook locally, use the smaller spatial extent and subset of the time series. 

If running on gadi, the the temporal and spatial extent can be increased.  

> **Note:** Landsat imagery is available from 1987 onwards. 

In [61]:
query_3 = {
        'lon': (134.24, 134.34),             # small test area - this works via vdi
        'lat': (-20.79, -20.86),             # small test area - this works via vdi
#        'lon': (132.07, 132.50),             # small test area - current test extent on vdi
#        'lat': (-20.31, -21.00),             # small test area - current test extent on vdi
#        'lon': (132.07, 135.36),             # full study area
#        'lat': (-20.31, -22.11),             # full study area
        'time':('2015-01', '2018-12'),       # subset of time-series
#        'time':('1987-01', '2018-12'),       # full time-series
        'output_crs': 'EPSG:28352',
         'resolution': (30, 30),
         'group_by': 'solar_day'
}

### Loading data

Load Landsat data from Collection 3 using .load_ard.

mask_dtype = np.float16 helps to keep the memory down, however, the data will need to be converted back to float34 later.

dask_chunks also helps to keep our memory use down and conduct the analysis in parallel. 'x' and 'y' define the pixel 'chunks' the dataset will be divided into. 


In [75]:
ds = dea_datahandling.load_ard(dc=dc_landsat3,
#        mask_dtype = np.float16,
        products=['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3'], 
        measurements=['nbart_red','nbart_nir','nbart_green',
                      'nbart_blue','nbart_swir_1','nbart_swir_2'],
        mask_contiguity='nbart_contiguity',
#        dask_chunks = {'x':500, 'y':500},
        **query_3)

Loading ga_ls5t_ard_3 data
    No data for ga_ls5t_ard_3
Loading ga_ls7e_ard_3 data
    Applying pixel quality/cloud mask
    Applying invalid data mask
    Applying contiguity mask
Loading ga_ls8c_ard_3 data
    Applying pixel quality/cloud mask
    Applying invalid data mask
    Applying contiguity mask
Combining and sorting data
    Returning 255 observations 


### Setting up automated file naming and location

In [76]:
#set variable for path to save files
savefilepath = '/g/data/zk34/ljg547/Outputs/'

# Set project naming convention. Start and end dates are reformated to remove '-'.
Proj = 'SSC_WD_'

ds_startDate = str(ds.isel(time=0).time.values)[0:10]
ds_startDate = str(ds_startDate[0:4] + f'{int(ds_startDate[6:7]):02d}' + 
              f'{int(ds_startDate[9:10]):02d}')

ds_endDate = str(ds.isel(time=-1).time.values)[0:10]
ds_endDate = str(ds_endDate[0:4] + f'{int(ds_endDate[6:7]):02d}' + 
              f'{int(ds_endDate[9:10]):02d}')

## Normalised Difference Vegetation Index (NDVI) 

Elaborate on NDVI here

In [77]:
# Calculate NDVI. NDVI is added to ds as a new band as shown in the below display
calculate_indices(ds,index = 'NDVI', collection = 'ga_ls_3',
        normalise = True, deep_copy = False)

<xarray.Dataset>
Dimensions:       (time: 255, x: 357, y: 272)
Coordinates:
  * y             (y) float64 7.684e+06 7.684e+06 ... 7.692e+06 7.692e+06
  * x             (x) float64 1.046e+06 1.046e+06 ... 1.056e+06 1.056e+06
  * time          (time) datetime64[ns] 2015-01-07T01:00:31.257836 ... 2018-12-25T00:55:54.633447
Data variables:
    nbart_red     (time, y, x) float32 nan nan nan nan ... 1986.0 1933.0 1894.0
    nbart_nir     (time, y, x) float32 nan nan nan nan ... 2958.0 2861.0 2832.0
    nbart_green   (time, y, x) float32 nan nan nan nan ... 1053.0 1055.0 1041.0
    nbart_blue    (time, y, x) float32 nan nan nan nan ... 545.0 546.0 532.0
    nbart_swir_1  (time, y, x) float32 nan nan nan nan ... 4233.0 4203.0 4095.0
    nbart_swir_2  (time, y, x) float32 nan nan nan nan ... 3655.0 3644.0 3592.0
    NDVI          (time, y, x) float32 nan nan nan ... 0.19357532 0.19847651
Attributes:
    crs:      EPSG:28352

### NDVI statistics for the full time series

Here we calculate mean and standard deviation of NDVI values for the full time series for each pixel. 

This provides information about the average 'greenness' and variation in 'greenness' over the long-term.

In [78]:
# Create a new NDVI DataArray 
ndvi = ds.NDVI

# Calculate mean NDVI through time for each pixel
mean_ndvi = ndvi.mean(dim='time')

# Calculate NDVI standard deviation through time for each pixel
std_ndvi = ndvi.std(dim='time')



### NDVI statistics for the defined dry season

Here we use the previously defined dry_months parameter to look at mean and standard deviation of NDVI during dry conditions over multiple years.

Looking at the same period of time over multiple years reduces the noise and highlights longer-term trends in vegetation greenness under dry conditions.

Vegetation with access to groundwater resources are hypothesised to have higher mean and lower standard deviation compared to vegetation with access to less reliable water sources (i.e. rain-fed).

mean_ndvi_dry can be viewed after each step to understand how each step compresses the time series data

In [79]:
# Group available NDVI time-steps into dry season months for later monthly averaging
ndvi_dryMonths = ndvi[ndvi['time.month'].isin(dry_months)]

In [80]:
# Calculate mean NDVI for each month in the dry period (through time and for each pixel)
mean_ndvi_dry = ndvi_dryMonths.groupby('time.month').mean(dim = 'time')

# Calculate mean NDVI for the dry period based on the mean monthly values for each month in the dry period
mean_ndvi_dry = mean_ndvi_dry.mean(dim = 'month')

In [81]:
# Calculate standard deviation in NDVI for each month in the dry period (through time and for each pixel)
std_ndvi_dry = ndvi_dryMonths.groupby('time.month').std(dim = 'time')

# Calculate mean standard deviation for the dry period based on the mean monthly values for each month in the dry period
std_ndvi_dry = std_ndvi_dry.mean(dim = 'month')

In [82]:
# Calculate the difference in NDVI standard deviation for two specific months. 0 = January; 7 = July
std_ndvi_diff1 = ndvi.groupby('time.month').std(dim = 'time').isel(month = 0)
std_ndvi_diff2 = ndvi.groupby('time.month').std(dim = 'time').isel(month = 7)
std_ndvi_diff = std_ndvi_diff1 - std_ndvi_diff2

### Exporting data
In order to use the datacube.helpers write_geotiff function to export a simple single-band, single time-slice geotiff the above xarray DataArrays need to be converted to xarray Datasets. 

We do this be using the xarray function .to_dataset. If you don't do this, the write_geotiff fucntion will return an error. 

We also need to reassign the coordinate reference system before the write_geotiff function will work. This is done by the .attrs function. We take the crs from the original imported data (ds). 

Each file will be exported as a geotiff and saved in the same directory as this notebook. It can be downloaded from this location to the GA network using FileZilla.


In [83]:
# Convert from float16 to float32
#arr = mean_ndvi_dry.astype(dtype='float32')

# Convert from DataArray to Dataset
#arr = arr.to_dataset(name='mean_NDVI_dry')
arr = mean_ndvi_dry.to_dataset(name='mean_NDVI_dry')

# Assign CRS from original DataArray
arr.attrs = ds.attrs

In [84]:
# Generating naming convention for dry season files based on Project area (Proj), specified dry season and time series start and end dates. 
fname = str(savefilepath + Proj + 'meanNDVI_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.tif')

In [85]:
# Writing data to file
write_geotiff(dataset = arr, filename = fname)

In [ ]:
# Creating an associated metadata file. w - writes, r - reads, a- appends
f = open(savefilepath + Proj + 'meanNDVI_' +
              ds_startDate + '_' + ds_endDate + '.txt','w')  

f.write("Mean NDVI for all months" + " from " + ds_startDate + 
      "-" + ds_endDate + "." + "\n" +
      "This product was derived from VegProducts_Export_Coll3_Dask.ipynb"
    )

f.close()

In [ ]:
# Convert from float16 to float32
#arr = std_ndvi.astype(dtype='float32')

# Convert from DataArray to Dataset
arr = std_ndvi.to_dataset(name='std_ndvi')

# Assign CRS from original DataArray
arr.attrs = ds.attrs

In [ ]:
# Generating naming convention for dry season files based on Project area (Proj), specified dry season and time series start and end dates. 
fname = str(savefilepath + Proj + 'stdNDVI_' +
              ds_startDate + '_' + ds_endDate + '.tif')

In [ ]:
# Writing data to file
write_geotiff(dataset=arr, filename=fname)

In [ ]:
# Creating an associated metadata file. w - writes, r - reads, a- appends
f = open(savefilepath + Proj + 'stdNDVI_' +
              ds_startDate + '_' + ds_endDate + '.txt','w')  

f.write("Standard deviation of NDVI for all months" + " from " + ds_startDate + 
      "-" + ds_endDate + "." + "\n" + 
      "Higher standard deviation suggests greater variation in vegetation greenness and therefore inferred water supply."
      "This product was derived from VegProducts_Export_Coll3_Dask.ipynb"
    )

f.close()

In [ ]:
# Convert from float16 to float32
#arr = std_ndvi_dry.astype(dtype='float32')

# Convert from DataArray to Dataset
arr = std_ndvi_dry.to_dataset(name='std_ndvi_dry')

# Assign CRS from original DataArray
arr.attrs = ds.attrs

In [ ]:
# Generating naming convention for dry season files based on Project area (Proj), specified dry season and time series start and end dates. 
fname = str(savefilepath + Proj + 'stdNDVI_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.tif')

In [ ]:
# Writing data to file
write_geotiff(dataset=arr, filename=fname)

In [ ]:
# Creating an associated metadata file. w - writes, r - reads, a- appends
f = open(savefilepath + Proj + 'stdNDVI_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.txt','w')  

f.write("Standard deviation of NDVI for the dry season(" + 
        str(dry_months[0]+1) + "-" + str(dry_months[-1]+1) + " month)" +  
        " from " + ds_startDate + "-" + ds_endDate + "." + "\n" + 
      "Higher standard deviation suggests greater variation in vegetation greenness and therefore inferred water supply."
      "This product was derived from VegProducts_Export_Coll3_Dask.ipynb"
    )

f.close()

In [ ]:
# Convert from float16 to float32
#arr = std_ndvi_diff.astype(dtype='float32')

# Convert from DataArray to Dataset
arr = std_ndvi_diff.to_dataset(name='std_ndvi_diff')

# Assign CRS from original DataArray
arr.attrs = ds.attrs

In [ ]:
# Generating naming convention for dry season files based on Project area (Proj), specified dry season and time series start and end dates. 
fname = str(savefilepath + Proj + 'stdNDVI_DiffJanAug_' +
              ds_startDate + '_' + ds_endDate + '.tif')

In [ ]:
# Writing data to file
write_geotiff(dataset=arr, filename=fname)

In [ ]:
# Creating an associated metadata file. w - writes, r - reads, a- appends
f = open(savefilepath + Proj + 'stdNDVI_DiffJanAug_' +
              ds_startDate + '_' + ds_endDate + '.txt','w')  

f.write("Comparison between NDVI standard deviation during the wet season (January) ad at the end of the dry season (August)." + "\n" 
      "Time series includes imagery from " + ds_startDate + "-" + ds_endDate + "." + "\n" + 
      "Where vegetation is accessing more reliable water sources (e.g. groundwater), residual standard deviation is low." + "\n"
      "This product was derived from VegProducts_Export_Coll3_Dask.ipynb"
    )

f.close()

In [ ]:
# Convert from float16 to float32
#arr = mean_ndvi_dry.astype(dtype='float32')

# Convert from DataArray to Dataset
arr = mean_ndvi_dry.to_dataset(name='mean_ndvi_dry')

# Assign CRS from original DataArray
arr.attrs = ds.attrs

In [ ]:
# Generating naming convention for dry season files based on Project area (Proj), specified dry season and time series start and end dates. 
fname = str(savefilepath + Proj + 'meanNDVI_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.tif')


In [ ]:
# Writing data to file
write_geotiff(dataset=arr, filename=fname)

In [ ]:
# Creating an associated metadata file. w - writes, r - reads, a- appends
f = open(savefilepath + Proj + 'meanTCW_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.txt','w')  

f.write("NDVI of dry period (" + str(dry_months[0]+1) + "-" + str(dry_months[-1]+1) + " month)" +  
      " from " + ds_startDate + "-" + ds_endDate + "." + "\n" +
      "NDVI_dry_mean is the mean value of NDVI over the dry months."+ "\n"
      "This product was derived from VegProducts_Export_Coll3.ipynb"
    )

f.close()

## Normalised Difference Wetness Index (NDWI)
Add text about NDWI

In [ ]:
# Calculate NDWI. NDWI is added to ds as a new band as shown in the below display
calculate_indices(ds,index = 'NDWI', collection = 'ga_ls_3',
        normalise = True, deep_copy = False)

### NDWI statistics for the defined dry season

Here we use the previously defined dry_months parameter to look at mean and standard deviation of NDWI during dry conditions over multiple years.

Looking at the same period of time over multiple years reduces the noise and highlights longer-term trends in vegetation wetness under dry conditions.

Vegetation with access to groundwater resources are hypothesised to have higher mean and lower standard deviation compared to vegetation with access to less reliable water sources (i.e. rain-fed).

mean_ndwi_dry can be viewed after each step to understand how each step compresses the time series data

In [ ]:
# Group available NDWI time-steps into dry season months for later monthly averaging
mean_ndwi_dry = ndwi[ndvi['time.month'].isin(dry_months)]

In [ ]:
# Calculate standard deviation in NDWI for each month in the dry period (through time and for each pixel)
mean_ndwi_dry = mean_ndwi_dry.groupby('time.month').mean(dim = 'time')

# Calculate mean standard deviation for the dry period based on the mean monthly values for each month in the dry period
mean_ndwi_dry = mean_ndwi_dry.mean(dim = 'month')

### Exporting data
In order to use the datacube.helpers write_geotiff function to export a simple single-band, single time-slice geotiff the above xarray DataArrays need to be converted to xarray Datasets. 

We do this be using the xarray function .to_dataset. If you don't do this, the write_geotiff fucntion will return an error. 

We also need to reassign the coordinate reference system before the write_geotiff function will work. This is done by the .attrs function. We take the crs from the original imported data (ds). 

Each file will be exported as a geotiff and saved in the same directory as this notebook. It can be downloaded from this location to the GA network using FileZilla.

In [ ]:
# Convert from float16 to float32
#arr = mean_ndwi_dry.astype(dtype='float32')

# Convert from DataArray to Dataset
arr = mean_ndwi_dry.to_dataset(name='mean_NDWI_dry')

# Assign CRS from original DataArray
arr.attrs = ds.attrs

In [ ]:
# Generating naming convention for dry season files based on Project area (Proj), specified dry season and time series start and end dates. 
fname = str(savefilepath + Proj + 'meanNDVI_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.tif')


In [ ]:
# Writing data to file
write_geotiff(dataset=arr, filename=fname)

In [ ]:
# Creating an associated metadata file. w - writes, r - reads, a- appends
f = open(savefilepath + Proj + 'meanLAI_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.txt','w')  


f.write("NDWI of dry period (" + str(dry_months[0]+1) + "-" + str(dry_months[-1]+1) + " month)" +  
      " from " + ds_startDate + "-" + ds_endDate + "." + "\n" +
      "NDWI_dry_mean is the mean value of NDVI over the dry months."+ "\n"
      "This product was derived from VegProducts_Export_Coll3.ipynb"
    )

f.close()

## Leaf Area Index (LAI)
Add text about LAI

In [ ]:
# Calculate LAI. LAI is added to ds as a new band as shown in the below display
calculate_indices(ds,index = 'LAI', collection = 'ga_ls_3',
        normalise = True, deep_copy = False)

In [ ]:
# Group available LAI time-steps into dry season months for later monthly averaging
mean_LAI_dry = ds.LAI['time.month'].isin(dry_months)

In [ ]:
# Calculate standard deviation in LAI for each month in the dry period (through time and for each pixel)
mean_LAI_dry = ds.LAI.groupby('time.month').mean(dim = 'time')

# Calculate mean standard deviation for the dry period based on the mean monthly values for each month in the dry period
mean_LAI_dry = mean_LAI_dry.mean(dim = 'month')

## Exporting data
In order to use the datacube.helpers write_geotiff function to export a simple single-band, single time-slice geotiff the above xarray DataArrays need to be converted to xarray Datasets. 

We do this be using the xarray function .to_dataset. If you don't do this, the write_geotiff fucntion will return an error. 

We also need to reassign the coordinate reference system before the write_geotiff function will work. This is done by the .attrs function. We take the crs from the original imported data (ds). 

Each file will be exported as a geotiff and saved in the same directory as this notebook. It can be downloaded from this location to the GA network using FileZilla.

In [ ]:
# Convert from float16 to float32
#arr = mean_LAI_dry.astype(dtype='float32')

# Convert from DataArray to Dataset
arr = mean_LAI_dry.to_dataset(name='mean_LAI_dry')

# Assign CRS from original DataArray
arr.attrs = ds.attrs

In [ ]:
# Generating naming convention for dry season files based on Project area (Proj), specified dry season and time series start and end dates. 
fname = str(savefilepath + Proj + 'meanLAI_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.tif')


In [ ]:
# Writing data to file
write_geotiff(dataset=arr, filename=fname)

In [ ]:
# Creating an associated metadata file. w - writes, r - reads, a- appends
f = open(savefilepath + Proj + 'meanLAI_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.txt','w')  

f.write("Leaf Area Index for the dry season (" + 
        str(dry_months[0]+1) + "-" + str(dry_months[-1]+1) + " month)" +  
        " from " + ds_startDate + "-" + ds_endDate + "." + "\n" +
        "LAI_dry_mean is the mean value of LAI over the dry months."+ "\n" +
        "This product was derived from VegProducts_Export_Coll3.ipynb"
    )

f.close()

## Tasselled Cap Index (TCI)
Add text about TCI inc diff bands

In [ ]:
# Calculate different band ratios. Each are Calculate NDWI. added to ds as a new band as shown in the below display
#Tasselled Cap Wetness
calculate_indices(ds,index = 'TCW', collection = 'ga_ls_3',
        normalise = True, deep_copy = False)

# Tasselled Cap Brightness
calculate_indices(ds,index = 'TCB', collection = 'ga_ls_3',
        normalise = True, deep_copy = False)

# Tasselled Cap Greenness
calculate_indices(ds,index = 'TCG', collection = 'ga_ls_3',
        normalise = True, deep_copy = False)

In [ ]:
# Tasselled Cap Wetness
# Group available TCW time-steps into dry season months for later monthly averaging
mean_TCW_dry = ds.TCW['time.month'].isin(dry_months)

# Calculate mean TCW for each month in the dry period (through time and for each pixel)
mean_TCW_dry = ds.TCW.groupby('time.month').mean(dim = 'time')

# Calculate mean standard deviation for the dry period based on the mean monthly values for each month in the dry period
mean_TCW_dry = mean_TCW_dry.mean(dim = 'month')

In [ ]:
# Tasselled Cap Brightness
# Group available TCB time-steps into dry season months for later monthly averaging
mean_TCB_dry = ds.TCB['time.month'].isin(dry_months)

# Calculate mean TCB for each month in the dry period (through time and for each pixel)
mean_TCB_dry = ds.TCB.groupby('time.month').mean(dim = 'time')

# Calculate mean standard deviation for the dry period based on the mean monthly values for each month in the dry period
mean_TCB_dry = mean_TCB_dry.mean(dim = 'month')

In [ ]:
# Tasselled Cap Greenness
# Group available TCG time-steps into dry season months for later monthly averaging
mean_TCG_dry = ds.TCG['time.month'].isin(dry_months)

# Calculate mean TCG for each month in the dry period (through time and for each pixel)
mean_TCG_dry = ds.TCG.groupby('time.month').mean(dim = 'time')

# Calculate mean standard deviation for the dry period based on the mean monthly values for each month in the dry period
mean_TCG_dry = mean_TCG_dry.mean(dim = 'month')

### Exporting data
In order to use the datacube.helpers write_geotiff function to export a simple single-band, single time-slice geotiff the above xarray DataArrays need to be converted to xarray Datasets. 

We do this be using the xarray function .to_dataset. If you don't do this, the write_geotiff fucntion will return an error. 

We also need to reassign the coordinate reference system before the write_geotiff function will work. This is done by the .attrs function. We take the crs from the original imported data (ds). 

Each file will be exported as a geotiff and saved in the same directory as this notebook. It can be downloaded from this location to the GA network using FileZilla.

In [ ]:
# Tasselled Cap Wetness
# Convert from float16 to float32
#arr = mean_TCW_dry.astype(dtype='float32')

# Convert from DataArray to Dataset
arr = mean_TCW_dry.to_dataset(name='mean_TCW_dry')

# Assign CRS from original DataArray
arr.attrs = ds.attrs

# Generating naming convention for dry season files based on Project area (Proj), specified dry season and time series start and end dates. 
fname = str(savefilepath + Proj + 'meanTCW_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.tif')

# Writing data to file
write_geotiff(dataset=arr, filename=fname)

# Creating an associated metadata file. w - writes, r - reads, a- appends
f = open(savefilepath + Proj + 'meanTCW_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.txt','w')  

f.write("Tasselled Cap Wetness for the dry season (" + 
        str(dry_months[0]+1) + "-" + str(dry_months[-1]+1) + " month)" +  
        " from " + ds_startDate + "-" + ds_endDate + "." + "\n" +
        "TCW_dry_mean is the mean value of TCW over the dry months."+ "\n" +
        "This product was derived from VegProducts_Export_Coll3_Dask.ipynb"
    )

f.close()

In [ ]:
# Tasselled Cap Brightness
# Convert from float16 to float32
#arr = mean_TCB_dry.astype(dtype='float32')

# Convert from DataArray to Dataset
arr = mean_TCB_dry.to_dataset(name='mean_TCB_dry')

# Assign CRS from original DataArray
arr.attrs = ds.attrs

# Generating naming convention for dry season files based on Project area (Proj), specified dry season and time series start and end dates. 
fname = str(savefilepath + Proj + 'meanTCB_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.tif')

# Writing data to file
write_geotiff(dataset=arr, filename=fname)

# Creating an associated metadata file. w - writes, r - reads, a- appends
f = open(savefilepath + Proj + 'meanTCB_DrySeason' +
              str(dry_months[0]+1) + 'to' + str(dry_months[-1]+1) +
              '_' + ds_startDate + '_' + ds_endDate + '.txt','w')  

f.write("Tasselled Cap Brightness for the dry season (" + 
        str(dry_months[0]+1) + "-" + str(dry_months[-1]+1) + " month)" +  
        " from " + ds_startDate + "-" + ds_endDate + "." + "\n" +
        "TCB_dry_mean is the mean value of TCB over the dry months."+ "\n" +
        "This product was derived from VegProducts_Export_Coll3.ipynb"
    )

f.close()

Must close the cluster after use, particularly if moving between notebooks

In [74]:
client.close()
client = None

cluster.close()
cluster = None

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** October 2019

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)